In [29]:
import pandas as pd
import useful_functions as uf
path_fill = 'data/2022/filled_trian.csv'
df_train = pd.read_csv('data/2022/train.csv', index_col=0).head(500)
df_train_filled = pd.read_csv(path_fill, index_col=0).head(500)
y_sample_500 = df_train_filled['LABEL']

Mode first,then go IQR filtering, then OneHot, then check corrlation, then 

IQR method

In [30]:
df_nu = uf.get_numerical_df(df_train_filled)

In [31]:
def IQR_filtering(df,U=0.88,L=0.22):
    """
    Optional
    ----------
    This function filters outliers using IQR method.

    Parameters
    ----------
    df : pd.Dataframe,
        This dataframe should be numerial only, if there is a string column, it will crash

    U : default is 0.88
        percentile of the upper bound, default is 0.88

    L : default is 0.22
        percentile of the lower bound, default is 0.22

    """

    for col in df.columns:
        Q1=df.loc[:,col].quantile(L)
        Q3=df.loc[:,col].quantile(U)
        IQR = Q3 - Q1
        lower = Q1-1.5*IQR
        upper = Q3+1.5*IQR
        con_cahnge = (df[col]<lower) | (df[col]>upper)
        def change(x):
            if x<lower:
                return lower
            elif x>upper:
                return upper
            else:
                return x
        df.loc[con_cahnge,col] = df.loc[con_cahnge,col].apply(change)
    return df


IQR filtering

In [32]:
df_train_filled.loc[:,IQR_filtering(df_nu).columns] = IQR_filtering(df_nu)

OneHot create_dummies()

In [33]:
df_train_filled=uf.create_dummies(df_train_filled)

now drop labels
uf.drop_high_corr_columns()---->list